In [1]:
import os 

In [4]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\Flight price prediction'

In [3]:
os.chdir('../')

In [5]:
from src.FlightPricePrediction.components.data_ingestion_02 import DataIngestion

In [6]:
import pandas as pd
data = DataIngestion().export_data_to_csv()
data.head()

[2025-03-13 16:27:13] - INFO - Entering the Data Ingestion module
[2025-03-13 16:27:13] - INFO - Connecting to the SQLite database
[2025-03-13 16:27:13] - INFO - Fetching all column headers from the table
[2025-03-13 16:27:13] - INFO - Fetching all rows of data from the table


[2025-03-13 16:27:13] - INFO - Writing data to CSV
[2025-03-13 16:27:14] - INFO - Exiting the Data Ingestion module


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [45]:
data.Airline.value_counts().head(5)

Airline
Jet Airways          4746
IndiGo               2564
Air India            2192
Multiple carriers    1543
SpiceJet             1026
Name: count, dtype: int64

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.1+ MB


In [47]:
# transform column to datetime
data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'])

C:\Users\nasir\AppData\Local\Temp\ipykernel_12560\1055296034.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'])


In [49]:
# Split the 'Arrival_Time' column into time and date parts
data[['Arrival_Time', 'Arrival_Date']] = data['Arrival_Time'].str.split(' ', expand=True, n=1)

# Keep only the time part
data['Arrival_Time'] = data['Arrival_Time'].fillna('00:00:00')

# Drop the 'Arrival_Date' column as it is not needed
data.drop(columns=['Arrival_Date'], inplace=True)

In [50]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0
1,Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25,19h,2 stops,No info,13882.0
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [51]:
# Function to convert duration in 'h m' format to total minutes
def convert_to_minutes(duration):
    hours = 0
    minutes = 0
    
    # Check if the duration contains hours and minutes (e.g., '3h 34m')
    if 'h' in duration and 'm' in duration:
        hours_part, minutes_part = duration.split('h')
        minutes_part = minutes_part.replace('m', '').strip()
        
        hours = int(hours_part.strip())  # Extract the hours
        minutes = int(minutes_part.strip())  # Extract the minutes
        
    # Check if the duration contains only hours (e.g., '3h')
    elif 'h' in duration:
        hours = int(duration.replace('h', '').strip())
        minutes = 0
    
    # Check if the duration contains only minutes (e.g., '34m')
    elif 'm' in duration:
        minutes = int(duration.replace('m', '').strip())
        
    # Convert hours to minutes and sum with the minutes
    total_minutes = (hours * 60) + minutes
    return total_minutes

# Apply the function to the Duration column and create a new column for total minutes
data['Duration(minute)'] = data['Duration'].apply(convert_to_minutes)

# Drop the original Duration column
data.drop(columns=['Duration'], inplace=True)

In [52]:
data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Duration(minute)
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10,non-stop,No info,3897.0,170
1,Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,2 stops,No info,7662.0,445
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25,2 stops,No info,13882.0,1140


In [53]:
data.drop(columns=['Additional_Info'], inplace=True)

In [54]:
data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Price,Duration(minute)
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10,non-stop,3897.0,170
1,Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,2 stops,7662.0,445
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25,2 stops,13882.0,1140


In [ ]:
# In Route column, replace ? to -> 

data['Route']= data['Route'].str.replace('?', '->')

0                      BLR -> DEL
1        CCU -> IXR -> BBI -> BLR
2        DEL -> LKO -> BOM -> COK
3               CCU -> NAG -> BLR
4               BLR -> NAG -> DEL
                   ...           
13349           CCU -> DEL -> BLR
13350                  CCU -> BLR
13351           DEL -> BOM -> COK
13352           DEL -> BOM -> COK
13353           DEL -> BOM -> COK
Name: Route, Length: 13354, dtype: object

In [57]:
data.head(3)    

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Price,Duration(minute)
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10,non-stop,3897.0,170
1,Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,2 stops,7662.0,445
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25,2 stops,13882.0,1140
